In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import argparse
import tarfile
import re

##impor  BMD files from directory
##TODO: combine bMD processing to single file, combine LPR processing to single file
import bmd_analysis_morpho as bmd
import bmd_analysis_LPR_7_PAH_t0_t239 as bmd_LPR
import format_LPR_input as format_LPR
import format_morpho_input as format_morpho

/Users/degn400/Desktop/Git_Repos/srpAnalytics/zfBmd
/Users/degn400/Desktop/Git_Repos/srpAnalytics/zfBmd


# Journey that a single morpho file takes

In [7]:
mfiles = './test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv'

morpho_input_csv_file_name_wide = mfiles.split("/")[-1] + "_wide_DNC_0_"+ "fd" + ".csv"

In [8]:
df_morph = pd.read_csv(mfiles, header = 0)

# Keep onloy relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
df_morph_select = df_morph.loc[:,columns_to_keep]

# Convert numerics to strings 
df_morph_select['plate.id'] = df_morph_select['plate.id'].astype(str)

In [95]:
df_morph = pd.read_csv(mfiles, header = 0)
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
all(col in df_morph.columns for col in relevant_columns) == False
print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)) 

The input file ./test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv must have the columns chemical.id, conc, plate.id, well, endpoint, value


In [9]:
df_morph_select.dtypes

chemical.id      int64
conc           float64
plate.id        object
well            object
endpoint        object
value          float64
dtype: object

In [68]:
df_reformatted = pd.DataFrame()

for chemical_index in pd.unique(df_morph['chemical.id']):

    print("chemical_index:" + str(chemical_index))
    #total_number_of_unique_chemicals += 1
    morph_data_chemical = df_morph_select.loc[df_morph['chemical.id'] == chemical_index,:]

    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        #total_number_of_chemical_plate_well += 1
        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]

        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in temp_df_grouped:

            ## JUSTIFICATION: 7 PAH dataset doesn't have "BRAI" endpoint.
            ## On the other hand, extracts/phase I,II have "BRAI" endpoint.
            if 'BRAI' not in temp_df.endpoint.values: # as 7 PAH
                try:
                    #            if(len(group.endpoint) == 14):
                    temp = pd.DataFrame( {
                        'chemical.id': pd.unique(temp_df['chemical.id']),
                        'plate.id': pd.unique(temp_df['plate.id']),
                        'well': pd.unique(temp_df['well']),
                        'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                        'conc': pd.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'SKIN': temp_df.value[temp_df.endpoint == 'SKIN'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,
                    }  )
                    df_reformatted = pd.concat([df_reformatted, temp])
                except:
                    print ("len(group.endpoint) != 14")
                    print ("chemical_plate_well:" + str(cpw))
            else: #as extracts
                temp = pd.DataFrame(
                    {
                    'chemical.id': pd.unique(temp_df['chemical.id']),
                    'plate.id': pd.unique(temp_df['plate.id']),
                    'well': pd.unique(temp_df['well']),
                    'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                    'conc': pd.unique(temp_df['conc']),
                    'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                    'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                    'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                    'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                    'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                    'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                    'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                    'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                    'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                    'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                    'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                    'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                    'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                    'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                    'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                    'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                    'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                    'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                    'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                    'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                    'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                    'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                    'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                    })
                df_reformatted = pd.concat([df_reformatted, temp])
#print ("total_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
#print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
#end_time = time.time()
#time_took = str(round((end_time-start_time), 1)) + " seconds"
#print ("Done, it took:"+str(time_took))

pd.set_option('display.max_columns', None)
print (df_reformatted.head())
#print ("df_reformatted.shape:" + str(df_reformatted.shape)) #(288, 19)

df_reformatted_DNC_0 = pd.DataFrame()

df_reformatted_DNC_0 = df_reformatted.loc[df_reformatted['DNC_'] == 0.0]
print ("df_reformatted_DNC_0.shape:" + str(df_reformatted_DNC_0.shape)) #(287, 19)

df_reformatted_DNC_0.to_csv("./Reformatted.csv", index=False)

chemical_index:3756
   chemical.id plate.id well chemical_plate_well   conc  AXIS  BRN_  CRAN  \
0         3756    20544  A01      3756_20544_A01  100.0   NaN   NaN   NaN   
0         3756    20544  A02      3756_20544_A02  100.0   0.0   0.0   0.0   
0         3756    20544  A03      3756_20544_A03  100.0   NaN   NaN   NaN   
0         3756    20544  A04      3756_20544_A04  100.0   0.0   0.0   0.0   
0         3756    20544  A05      3756_20544_A05  100.0   0.0   0.0   0.0   

   DNC_  DP24  EDEM  LTRK  MO24  MORT  MUSC  NC__  SKIN  SM24  TCHR  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
df_reformatted_DNC_0.shape:(287, 19)


In [98]:
# Pivot long format table wider, and sum up the ones 
pivot_wider = df_morph_select.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], values = ['value'], aggfunc = 'first')

# Convert to a clean pandas dataframe 
pivot_wider = pd.DataFrame(pivot_wider.to_records())

# Clean up the column names 
new_colnames = []

# Fix column names with the extra 'value' annotation. Otherwise, ignore.
for name in pivot_wider.columns:
    if "value" in name:
        new_colnames.append(re.sub("\\(value,|\\)", "", re.sub("'", "", name)).strip())
    else:
        new_colnames.append(name)

# Rename columns
pivot_wider = pivot_wider.set_axis(new_colnames, axis = 1, inplace = False)

In [114]:
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
for x in ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']:
    relevant_columns.append(x)
relevant_columns

['chemical.id',
 'conc',
 'plate.id',
 'well',
 'endpoint',
 'value',
 <generator object <genexpr> at 0x161961040>]

In [118]:
df_morph.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], 
                                       values = ['value'], aggfunc = 'first')

value                                          \
endpoint                         AXIS BRN_ CRAN DNC_ DP24 EDEM LTRK MO24 MORT   
chemical.id plate.id well conc                                                  
3756        20544    A01  100.0   NaN  NaN  NaN  0.0  NaN  NaN  NaN  1.0  NaN   
                     A02  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     A03  100.0   NaN  NaN  NaN  0.0  NaN  NaN  NaN  1.0  NaN   
                     A04  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     A05  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
            20625    H08  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     H09  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     H10  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     H11  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     H12  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                          
endpoint                        MUSC NC__ SKIN SM24 TCHR  
chemical.id plate.id well conc                            
3756        20544    A01  100.0  NaN  NaN  NaN  NaN  NaN  
                     A02  100.0  0.0  0.0  0.0  0.0  0.0  
                     A03  100.0  NaN  NaN  NaN  NaN  NaN  
                     A04  100.0  0.0  0.0  0.0  0.0  0.0  
                     A05  100.0  0.0  0.0  0.0  0.0  0.0  
...                              ...  ...  ...  ...  ...  
            20625    H08  0.0    0.0  0.0  0.0  0.0  0.0  
                     H09  0.0    0.0  0.0  0.0  0.0  0.0  
                     H10  0.0    0.0  0.0  0.0  0.0  0.0  
                     H11  0.0    0.0  0.0  0.0  0.0  0.0  
                     H12  0.0    0.0  0.0  0.0  0.0  0.0  

[288 rows x 14 columns]

# Morpho Step 1: Format File

In [79]:
# This is the format_morpho_input function

###################################
## PIVOT WIDER AND CLEAN RESULTS ##
###################################

# Read morphology file 
df_morph = pd.read_csv('./test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv', header = 0)

# List relevant column names
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']

# The input file must absolutely have these columns, no exceptions 
if all(col in df_morph.columns for col in relevant_columns) == False:
    sys.exit(print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)))

# Keep only relevant columns
df_morph = df_morph.loc[:,relevant_columns]

# Pivot long format table wider, and sum up the ones 
pivot_wider = df_morph.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], 
                                   values = ['value'], aggfunc = 'first')

# Convert to a clean pandas dataframe 
pivot_wider = pd.DataFrame(pivot_wider.to_records())

# Clean up the column names 
new_colnames = []

# Fix column names with the extra 'value' annotation. Otherwise, ignore.
for name in pivot_wider.columns:
    if "value" in name:
        new_colnames.append(re.sub("\\(value,|\\)", "", re.sub("'", "", name)).strip())
    else:
        new_colnames.append(name)

# Rename columns
pivot_wider = pivot_wider.set_axis(new_colnames, axis = 1, inplace = False)

# Get the relevant endpoints, which is different for brain samples 
if "BRAI" in pivot_wider.columns:
    relevant_endpoints = ['AXIS', 'BRAI', 'CFIN', 'CIRC', 'DNC_', 'DP24', 'EYE_', 'JAW_', 'MO24', 
                          'MORT', 'NC24', 'NC__', 'OTIC', 'PE__', 'PFIN', 'PIG_', 'SM24', 'SNOU', 
                          'SOMI', 'SWIM', 'TRUN', 'TR__', 'YSE_']
else:
    relevant_endpoints = ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 
                          'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']

# Add relevant endpoints to the list of relevant columns, with value and endpints removed
relevant_columns.remove("endpoint")
relevant_columns.remove("value")
for x in relevant_endpoints:
    relevant_columns.append(x)

# Subset by those relevant columns 
pivot_wider = pivot_wider.loc[:,relevant_columns]

###########################
## ADD MISSING ENDPOINTS ##
###########################

# Create an adding function that does not go beyond one --> just take max 
def sum_max_1(subset_df):
    newsum = subset_df.sum(axis=1, skipna = True, min_count = 1)
    newsum[newsum > 1] = 1
    return(newsum)

if 'BRAI' not in pivot_wider.columns: 
    
    # 1. Add any effect at 24hrs (combination of MO24, DP24 and SM24) 
    pivot_wider['ANY24'] = sum_max_1(pivot_wider[['MO24','DP24','SM24']])
    
    # 2. Any effect within 5 days (combination of all measurements at both time points)
    pivot_wider['ANY120'] = sum_max_1(pivot_wider[['AXIS', 'BRN_', 'CRAN', 'EDEM', 'LTRK', 'MORT', 'MUSC', 
                                                   'NC__', 'SKIN', 'TCHR', 'ANY24']])

    # 3. Total mortality (MO24 + MORT) 
    pivot_wider['TOT_MORT'] = sum_max_1(pivot_wider[['MO24','MORT']])

    # 4. Any effect except mortality (#2 minus MO24 and MORT)
    pivot_wider['ALL_BUT_MORT'] = sum_max_1(pivot_wider[['AXIS', 'BRN_', 'CRAN', 'DP24', 'EDEM', 'LTRK', 
                                                         'MUSC', 'NC__', 'SKIN', 'SM24', 'TCHR']])
    
else:
    pivot_wider['ANY24'] = sum_max_1(pivot_wider[['MO24','DP24','SM24','NC24']])
    pivot_wider['ANY120'] = sum_max_1(pivot_wider[['MORT', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 
                                   'PE__', 'BRAI', 'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC', 
                                   'TRUN', 'SWIM', 'NC__', 'TR__', 'ANY24']])
    pivot_wider['TOT_MORT'] = sum_max_1([['MO24','MORT']])
    pivot_wider['ALL_BUT_MORT'] = sum_max_1(pivot_wider[['DP24','SM24','NC24', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 
                                         'PE__', 'BRAI', 'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC','TRUN', 'SWIM', 'NC__', 
                                         'TR__']])
    pivot_wider['BRN_'] = sum_max_1(pivot_wider[['BRAI','OTIC','PFIN']])
    pivot_wider['CRAN'] = sum_max_1(pivot_wider[['EYE_', 'SNOU', 'JAW_']])
    pivot_wider['EDEM'] = sum_max_1(pivot_wider[['YSE_','PE__']])
    pivot_wider['LTRK'] = sum_max_1( pivot_wider[['TRUN','CFIN']])
    pivot_wider['MUSC'] = sum_max_1(pivot_wider[['CIRC','SWIM','SOMI']])
    pivot_wider['SKIN'] = pivot_wider[['PIG_']]
    pivot_wider['TCHR'] = pivot_wider[['TR__']]

pivot_wider
pivot_wider.to_csv("/Users/degn400/Desktop/srpTargets/zfBmd/pivot_wider.csv", index = False)

# Format Step 2: Generate Dose Response

In [102]:
def test_dose_response(end_point):
    data_ep_cid = pivot_wider
    dose_response = pd.DataFrame(columns = ['dose', 'num_affected', 'frac_affect', 'num_embryos', 'tot_wells'])

    for plate_id in np.unique(data_ep_cid['plate.id']):

        # Count number of wells corresponding to negative controls
        data_ep_cid_plate = data_ep_cid.loc[data_ep_cid['plate.id'] == plate_id]
        neg_ctrl_wells = data_ep_cid_plate.loc[data_ep_cid_plate['conc'] == 0]

        num_neg_ctrl_wells = neg_ctrl_wells.shape[0]
        num_neg_ctrl_hits = (neg_ctrl_wells[end_point]).sum(axis=0,skipna=True,max_count=1)
        num_nonnan_wells_ctrl = sum(~np.isnan(neg_ctrl_wells[end_point]))

        if(num_neg_ctrl_hits > 0.5*num_nonnan_wells_ctrl): # new criterion
            data_ep_cid = data_ep_cid[data_ep_cid['plate.id'] != plate_id]


    for concentration_id in np.unique(data_ep_cid['conc']):
        data_ep_cid_concs = data_ep_cid.loc[(data_ep_cid['conc'] == concentration_id)]

        # Get total number of wells for a given concentration
        tot_wells = len(data_ep_cid_concs[end_point])
        num_nonnan_wells = sum(~np.isnan(data_ep_cid_concs[end_point]))
        num_affected = (data_ep_cid_concs[end_point]).sum(axis=0,skipna=True,min_count=1)
        if(num_nonnan_wells == 0):
            fraction_affected = np.nan
        else:
            fraction_affected = num_affected / num_nonnan_wells
        dose_response = dose_response.append({'dose': concentration_id, 'num_affected': num_affected , 'frac_affect': fraction_affected, 'num_embryos': num_nonnan_wells, 'tot_wells': tot_wells}, ignore_index = True)


    # First get rid of nan values
    dose_response = dose_response.dropna()

    #''' dose  num_affected  frac_affect  num_embryos  tot_wells
    #0   0.0         0.0     0.000000         26.0       32.0
    #1   0.1         1.0     0.032258         31.0       32.0
    #2   0.5         1.0     0.062500         16.0       32.0
    #'''
    delete_count = 0


    for dr_index in range(dose_response.shape[0]):
        dr_index_original = dr_index
        dr_index = dr_index - delete_count

        if((dose_response.iloc[dr_index].num_embryos) < (0.25*(dose_response.iloc[dr_index].tot_wells))):
            dose_response = dose_response[dose_response.index != dr_index_original]
            delete_count+=1

    return(dose_response)

test_dose_response("BRN_")

TypeError: sum() got an unexpected keyword argument 'max_count'

In [59]:
# Before calculating metrics, we remove any cases where the number of negative control hits is greater 
data_ep_cid_plate = data_ep_cid.loc[data_ep_cid['plate.id'] == plate_id]
neg_ctrl_wells = data_ep_cid_plate.loc[data_ep_cid_plate['conc'] == 0]
num_neg_ctrl_wells = neg_ctrl_wells.shape[0]

for x in ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 'MUSC', 'NC__', 'SKIN','SM24', 'TCHR', 'ANY24', 'ANY120', 'TOT_MORT', 'ALL_BUT_MORT']:
    num_neg_ctrl_hits = (neg_ctrl_wells[x]).sum(axis=0,skipna=True,min_count=1)
    num_nonnan_wells_ctrl = sum(~np.isnan(neg_ctrl_wells[end_point]))
    #print(x)
    #print(num_neg_ctrl_hits > 0.5*num_nonnan_wells_ctrl)

In [97]:
num_embryos = pivot_wider.drop(["plate.id", "well"], 1).groupby(by = ["chemical.id", "conc"], dropna = True).count()
num_embryos

AXIS  BRN_  CRAN  DNC_  DP24  EDEM  LTRK  MO24  MORT  \
chemical.id conc                                                           
3756        0.00      32    32    32    36    32    32    32    36    32   
            2.16      34    34    34    36    34    34    34    36    34   
            6.80      31    31    31    36    35    31    31    36    35   
            14.70     31    31    31    36    33    31    31    36    33   
            31.60     28    28    28    36    33    28    28    36    33   
            56.20     25    25    25    36    31    25    25    35    31   
            75.00     29    29    29    36    30    29    29    36    30   
            100.00    30    30    30    36    32    30    30    36    32   

                    MUSC  NC__  SKIN  SM24  TCHR  ANY24  ANY120  TOT_MORT  \
chemical.id conc                                                            
3756        0.00      32    32    32    32    32     36      36        36   
            2.16      34    34    34    34    34     36      36        36   
            6.80      31    31    31    35    31     36      36        36   
            14.70     31    31    31    33    31     36      36        36   
            31.60     28    28    28    33    28     36      36        36   
            56.20     25    25    25    31    25     35      35        35   
            75.00     29    29    29    30    29     36      36        36   
            100.00    30    30    30    32    30     36      36        36   

                    ALL_BUT_MORT  
chemical.id conc                  
3756        0.00              32  
            2.16              34  
            6.80              35  
            14.70             33  
            31.60             33  
            56.20             31  
            75.00             30  
            100.00            32

In [98]:
num_affected = pivot_wider.drop("plate.id", 1).groupby(by = ["chemical.id", "conc"], dropna = True).sum()
num_affected

AXIS  BRN_  CRAN  DNC_  DP24  EDEM  LTRK  MO24  MORT  \
chemical.id conc                                                           
3756        0.00     2.0   0.0   2.0   0.0   0.0   2.0   1.0   4.0   0.0   
            2.16     0.0   0.0   0.0   0.0   1.0   0.0   0.0   2.0   0.0   
            6.80     1.0   0.0   1.0   0.0   0.0   1.0   0.0   1.0   4.0   
            14.70    3.0   0.0   3.0   0.0   0.0   3.0   1.0   3.0   2.0   
            31.60    1.0   0.0   1.0   0.0   1.0   1.0   0.0   3.0   5.0   
            56.20    2.0   0.0   2.0   1.0   2.0   2.0   0.0   4.0   6.0   
            75.00    5.0   0.0   5.0   0.0   4.0   5.0   3.0   6.0   1.0   
            100.00   1.0   0.0   1.0   0.0   2.0   1.0   0.0   4.0   2.0   

                    MUSC  NC__  SKIN  SM24  TCHR  ANY24  ANY120  TOT_MORT  \
chemical.id conc                                                            
3756        0.00     2.0   0.0   0.0   0.0   0.0    4.0     6.0       4.0   
            2.16     0.0   0.0   0.0   0.0   0.0    3.0     3.0       2.0   
            6.80     1.0   0.0   0.0   0.0   0.0    1.0     6.0       5.0   
            14.70    3.0   0.0   0.0   0.0   0.0    3.0     8.0       5.0   
            31.60    1.0   0.0   0.0   0.0   0.0    4.0     9.0       8.0   
            56.20    2.0   0.0   0.0   0.0   0.0    6.0    12.0      10.0   
            75.00    5.0   0.0   0.0   0.0   1.0   10.0    13.0       7.0   
            100.00   1.0   0.0   0.0   0.0   0.0    6.0     7.0       6.0   

                    ALL_BUT_MORT  
chemical.id conc                  
3756        0.00             2.0  
            2.16             1.0  
            6.80             1.0  
            14.70            3.0  
            31.60            2.0  
            56.20            4.0  
            75.00            6.0  
            100.00           3.0

In [99]:
frac_affected = num_affected / num_embryos
frac_affected

AXIS  BRN_      CRAN      DNC_      DP24      EDEM  \
chemical.id conc                                                             
3756        0.00    0.062500   0.0  0.062500  0.000000  0.000000  0.062500   
            2.16    0.000000   0.0  0.000000  0.000000  0.029412  0.000000   
            6.80    0.032258   0.0  0.032258  0.000000  0.000000  0.032258   
            14.70   0.096774   0.0  0.096774  0.000000  0.000000  0.096774   
            31.60   0.035714   0.0  0.035714  0.000000  0.030303  0.035714   
            56.20   0.080000   0.0  0.080000  0.027778  0.064516  0.080000   
            75.00   0.172414   0.0  0.172414  0.000000  0.133333  0.172414   
            100.00  0.033333   0.0  0.033333  0.000000  0.062500  0.033333   

                        LTRK      MO24      MORT      MUSC  NC__  SKIN  SM24  \
chemical.id conc                                                               
3756        0.00    0.031250  0.111111  0.000000  0.062500   0.0   0.0   0.0   
            2.16    0.000000  0.055556  0.000000  0.000000   0.0   0.0   0.0   
            6.80    0.000000  0.027778  0.114286  0.032258   0.0   0.0   0.0   
            14.70   0.032258  0.083333  0.060606  0.096774   0.0   0.0   0.0   
            31.60   0.000000  0.083333  0.151515  0.035714   0.0   0.0   0.0   
            56.20   0.000000  0.114286  0.193548  0.080000   0.0   0.0   0.0   
            75.00   0.103448  0.166667  0.033333  0.172414   0.0   0.0   0.0   
            100.00  0.000000  0.111111  0.062500  0.033333   0.0   0.0   0.0   

                        TCHR     ANY24    ANY120  TOT_MORT  ALL_BUT_MORT  
chemical.id conc                                                          
3756        0.00    0.000000  0.111111  0.166667  0.111111      0.062500  
            2.16    0.000000  0.083333  0.083333  0.055556      0.029412  
            6.80    0.000000  0.027778  0.166667  0.138889      0.028571  
            14.70   0.000000  0.083333  0.222222  0.138889      0.090909  
            31.60   0.000000  0.111111  0.250000  0.222222      0.060606  
            56.20   0.000000  0.171429  0.342857  0.285714      0.129032  
            75.00   0.034483  0.277778  0.361111  0.194444      0.200000  
            100.00  0.000000  0.166667  0.194444  0.166667      0.093750

AXIS  BRN_  CRAN  DNC_  DP24  EDEM  LTRK  MO24  MORT  \
chemical.id conc                                                           
3756        0.00     2.0   0.0   2.0   0.0   0.0   2.0   1.0   4.0   0.0   
            2.16     0.0   0.0   0.0   0.0   1.0   0.0   0.0   2.0   0.0   
            6.80     1.0   0.0   1.0   0.0   0.0   1.0   0.0   1.0   4.0   
            14.70    3.0   0.0   3.0   0.0   0.0   3.0   1.0   3.0   2.0   
            31.60    1.0   0.0   1.0   0.0   1.0   1.0   0.0   3.0   5.0   
            56.20    2.0   0.0   2.0   1.0   2.0   2.0   0.0   4.0   6.0   
            75.00    5.0   0.0   5.0   0.0   4.0   5.0   3.0   6.0   1.0   
            100.00   1.0   0.0   1.0   0.0   2.0   1.0   0.0   4.0   2.0   

                    MUSC  NC__  SKIN  SM24  TCHR  ANY24  ANY120  TOT_MORT  \
chemical.id conc                                                            
3756        0.00     2.0   0.0   0.0   0.0   0.0    4.0    13.0       4.0   
            2.16     0.0   0.0   0.0   0.0   0.0    3.0     3.0       2.0   
            6.80     1.0   0.0   0.0   0.0   0.0    1.0     9.0       5.0   
            14.70    3.0   0.0   0.0   0.0   0.0    3.0    18.0       5.0   
            31.60    1.0   0.0   0.0   0.0   0.0    4.0    13.0       8.0   
            56.20    2.0   0.0   0.0   0.0   0.0    6.0    20.0      10.0   
            75.00    5.0   0.0   0.0   0.0   1.0   10.0    35.0       7.0   
            100.00   1.0   0.0   0.0   0.0   0.0    6.0    12.0       6.0   

                    ALL_BUT_MORT  
chemical.id conc                  
3756        0.00             9.0  
            2.16             1.0  
            6.80             4.0  
            14.70           13.0  
            31.60            5.0  
            56.20           10.0  
            75.00           28.0  
            100.00           6.0

In [42]:
tot_wells = pivot_wider.drop("plate.id", 1).groupby(by = ["chemical.id", "conc"]).size()
tot_wells

chemical.id  conc  
3756         0.00      36
             2.16      36
             6.80      36
             14.70     36
             31.60     36
             56.20     36
             75.00     36
             100.00    36
dtype: int64